In [84]:
import dill
import pandas as pd
import datetime 
import plotly.express as px

In [2]:
print("Pandas version", pd.__version__)

Pandas version 1.2.4


In [4]:
df = dill.load(open('Prices/AZN.pkd', 'rb'))

In [5]:
df

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
1994-12-01,39.13,39.3800,39.13,39.3800,12500,2.656528,2.673500,2.656528,2.673500,75000,0.0,1.0
1994-12-02,38.75,39.1300,38.75,39.1300,1500,2.630730,2.656528,2.630730,2.656528,9000,0.0,1.0
1994-12-05,38.75,38.9400,38.75,38.8700,41500,2.630730,2.643629,2.630730,2.638877,249000,0.0,1.0
1994-12-06,38.87,38.8700,38.75,38.8700,15800,2.638877,2.638877,2.630730,2.638877,94800,0.0,1.0
1994-12-07,39.00,39.2800,38.75,38.7500,56600,2.647702,2.666711,2.630730,2.630730,339600,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-13,50.88,51.2500,50.67,50.7400,11452845,50.880000,51.250000,50.670000,50.740000,11452845,0.0,1.0
2021-01-14,51.29,51.8000,51.17,51.4700,11521679,51.290000,51.800000,51.170000,51.470000,11521679,0.0,1.0
2021-01-15,51.34,51.5600,51.13,51.2900,9523952,51.340000,51.560000,51.130000,51.290000,9523952,0.0,1.0


In [44]:
def find_entry(ticker, date, ndays):
    df = dill.load(open('Prices/' + ticker + '.pkd', 'rb'))
    df = df['close'].reset_index()
    
    df = df[df['date'] >= date - datetime.timedelta(days = ndays)]
    df = df[df['date'] <= date + datetime.timedelta(days = ndays)]
    df = df.reset_index()
    
    return df.iloc[df['close'].idxmin()]

In [45]:
def find_exit(ticker, date, ndays):
    df = dill.load(open('Prices/' + ticker + '.pkd', 'rb'))
    df = df['close'].reset_index()
    
    df = df[df['date'] >= date - datetime.timedelta(days = ndays)]
    df = df[df['date'] <= date + datetime.timedelta(days = ndays)]
    df = df.reset_index()
    
    return df.iloc[df['close'].idxmax()]

In [71]:
find_entry('AZN', datetime.datetime(2020, 1, 1), 2)

index                   6314
date     2019-12-31 00:00:00
close                  49.86
Name: 1, dtype: object

In [162]:
def calculate_profit_by_period(ticker, date, ndays = 3, period_length = 30, num_periods = 13):
    profits = []
    original_date = date
    
    _, entry_date, initial_price, = find_entry('AZN', date, ndays)
    
    for i in range(num_periods):
        date = date + datetime.timedelta(days = period_length)
        _, exit_date, new_price = find_exit('AZN', date, ndays)
        profits.append({'Period' : i + 1, 
                        'Percent Profit (%)' : (round((new_price - initial_price) / initial_price, 4)) * 100,  
                        'Study Date' : original_date.strftime('%Y-%m-%d'), 
                        'Entry Date' : entry_date.strftime('%Y-%m-%d'), 
                        'Exit Date' : exit_date.strftime('%Y-%m-%d')})
        
    return pd.DataFrame(profits)

In [163]:
# Fill this in with the actual study dates
df = pd.DataFrame()
study_dates = [datetime.datetime(2015, 1, 1), datetime.datetime(2016, 1, 1), datetime.datetime(2017, 1, 1), datetime.datetime(2018, 1, 1), datetime.datetime(2019, 1, 1)]
for date in study_dates:
    df = df.append(calculate_profit_by_period('AZN', date))

    
    
fig = px.line(df, x = 'Period', y = 'Percent Profit (%)', color='Study Date', hover_data = ['Entry Date', 'Exit Date'],  title="AZN Profit After Studies Over 30 Day Periods")
fig.update_xaxes(type='category')
fig.show()